Import libraries

In [12]:
import os
import cv2
import numpy as np

import face_recognition

Set the path

In [13]:
path = 'images'

Empty lists

In [14]:
images = []
classnames = []
mylist = os.listdir(path)

Loop through image folder

In [15]:
for cl in mylist:
    currentImage = cv2.imread(os.path.join(path, cl))
    images.append(currentImage)
    classnames.append(os.path.splitext(cl)[0])

Function to find encodings

In [16]:
def findEncodings(images):
    encodelist = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodelist.append(encode)
    return encodelist

Get encodings of knwon face

In [17]:
encodelistknownfaces = findEncodings(images)

Webcam test

In [20]:
cap = cv2.VideoCapture(0)
while True:
    sucess, img = cap.read()
    imagesmall = cv2.resize(img,(0,0),None,0.25,0.25)

    faceInFrame = face_recognition.face_locations(imagesmall)
    encodeFrame = face_recognition.face_encodings(imagesmall, faceInFrame)

    for encodeFace, faceLoc in zip(encodeFrame, faceInFrame):
        matchs = face_recognition.compare_faces(encodelistknownfaces, encodeFace)
        faceDistance  = face_recognition.face_distance(encodelistknownfaces, encodeFace)
        matchIndex = np.argmin(faceDistance)

        if matchs[matchIndex] and faceDistance[matchIndex] < 0.6:
            name = classnames[matchIndex].upper()
            color = (0,255, 0)
        else:
            name = "UNKNOWN"
            color = (0, 0, 255)

        y1, x2, y2, x1 = faceLoc
        y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4
        cv2.rectangle(img, (x1, y2-35), (x2, y2), color, cv2.FILLED)
        cv2.putText(img, name, (x1+6, y2-6), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

    cv2.imshow('Face Recognition', img)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
